<a href="https://colab.research.google.com/github/MohamedAbdelAzeem/Vehicle_Detection/blob/main/phase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import cv2
import os
import glob
import matplotlib.pyplot as plt
from moviepy.editor import *

### Load Weights and CFG

In [ ]:
#weights_path = os.path.join("yolo","C:\\Users\\THE WHALE\\Vechile_Detection_YOLO_GPU\\yolov3.weights")
#cfg_path = os.path.join("yolo","C:\\Users\\THE WHALE\\Vechile_Detection_YOLO_GPU\\yolov3.cfg")
labels = open('coco.names').read().strip().split('\n')     #Load COCO Dataset
print(labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


### Load Neural Netowrk

In [ ]:
import cv2
net = cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights')

In [ ]:
##Getting Layer Names and Output Layers
names = net.getLayerNames()
outputlayers = list(net.getUnconnectedOutLayersNames())  
print(outputlayers)

['yolo_82', 'yolo_94', 'yolo_106']


### Inference

In [ ]:
def process(image):
    boxes = []
    confidences = []
    classIDs = []

    H,W = image.shape[:2]   #height and width of the image
    blob = cv2.dnn.blobFromImage(image,1/255.0,(416,416),crop=False,swapRB=False)  #preprocessing the image
    net.setInput(blob)  #input the image to the network
    layers_output = net.forward(outputlayers)
    for output in layers_output:
      for detection in output:
          scores = detection[5:]
          classID = np.argmax(scores)   #getting ID of the class that has maximum score
          confidence = scores[classID]
          if confidence > 0.75: 
              box = detection[:4] * np.array([W,H,W,H])    #Getting center,width and height of the box
               bx,by,bw,bh = box.astype("int")
                x = int(bx-(bw/2))
                y = int(by-(bh/2))
                
                boxes.append([x,y,int(bw),int(bh)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.75,0.6)  #Filtering boxes using non maximum suppression
    if len(indexes) > 0:
        for i in indexes.flatten():
            x,y = [boxes[i][0],boxes[i][1]]         # top left corner point 
            w,h = [boxes[i][2],boxes[i][3]]         #width and height of the box
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,139,139),2)
            cv2.putText(image,"{}:{}".format(labels[classIDs[i]],confidences[i]),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,139,139),2)
    return image


In [ ]:
clip = VideoFileClip("project_video.mp4")
start = time.time()
final = clip.fl_image(process).subclip(0,30)
%time final.write_videofile("results\output_project_video.mp4", audio=False)

Moviepy - Building video results\output_project_video.mp4.
Moviepy - Writing video results\output_project_video.mp4



Moviepy - Done !
Moviepy - video ready results\output_project_video.mp4
Wall time: 3min 50s
